### MAST30034: Applied Data Science Project 2
---
# Naive Ranking: Unweighted Average Rank

Calculate the unweights average rank over all merchants (without segmentation).

In [1]:
# useful imports
import pandas as pd
import numpy as np
import os
import re

os.chdir("../scripts")
import utilities.rank_utilities as RANK

In [2]:
import json
# read in the segments that Oliver defined
segments = json.load(open('../data/curated/segments.json'))
segments

{'leisure goods and services': [11944993446,
  13467303030,
  13823979247,
  14081759429,
  14148282104,
  14809247463,
  15253672771,
  15560455575,
  17507773571,
  17541067926,
  20422223827,
  20445424481,
  23541969839,
  24164521701,
  27326652377,
  28034382339,
  29917527785,
  32610248934,
  32897338221,
  37470904997,
  38700038932,
  38734016504,
  39759375662,
  40507108040,
  41333655501,
  42399949944,
  43698901053,
  43835684204,
  44461622420,
  45151159881,
  45899477665,
  46331355995,
  46877655150,
  47484801038,
  48294754935,
  48625977738,
  48854820141,
  49418428210,
  49460691968,
  49891706470,
  52827561456,
  53378405666,
  55179323922,
  57093429717,
  57798993346,
  58729395250,
  60616659469,
  60654402457,
  62170730910,
  63948574402,
  64185141673,
  66118124566,
  66257020651,
  66610548417,
  68514534266,
  68656010159,
  69655310165,
  70088042348,
  70172340121,
  71305424518,
  73004400052,
  73841664453,
  74753274268,
  75239540583,
  79306976

In [3]:
# create a mapping df real quick
rows = []

for seg, abn_list in segments.items():
    if seg == 'repair services': continue
    for abn in abn_list:
        rows.append({'segment': seg, 'merchant_abn': abn})

segments_df = pd.DataFrame(rows)
segments_df

,segment,merchant_abn
0,leisure goods and services,11944993446
1,leisure goods and services,13467303030
2,leisure goods and services,13823979247
3,leisure goods and services,14081759429
4,leisure goods and services,14148282104
...,...,...
2899,tech and telecom,91062920626
2900,tech and telecom,92594285117
2901,tech and telecom,93740493847
2902,tech and telecom,93932766844


In [4]:
# read the curated data
merchant_df = pd.read_parquet('../data/curated/final_merchant_statistics')

In [5]:
# filter for only merchants in our segments of choice
merchant_df = merchant_df[merchant_df['merchant_abn'].isin(segments_df['merchant_abn'])]

In [6]:
merchant_df.head(5)

,merchant_abn,name,tags,tag,revenue_level,take_rate,tot_dollar_value,avg_dollar_value,stddev_dollar_value,tot_discounted_value,...,commission_stddev_tot_dollar_value_daily,commission_avg_tot_discounted_value_daily,commission_stddev_tot_discounted_value_daily,numd_sa2_code,median_weekly_income,returning_customers,unique_customers,avg_tot_dollar_value_customers,stddev_tot_dollar_value_customers,vip_customers
0,10023283211,Felis Limited,"((furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",e,0.18,7.032777e+05,215.663205,138.039260,6.554602e+05,...,0.002111,1.946912,0.001963,1717.0,794.816023,11.0,3032.0,231.951752,155.424439,137.0
1,10346855916,Odio Institute,"((equipment, tool, furniture, and appliance r...","equipment, tool, furniture, and appliance ren...",b,3.57,6.775178e+03,1129.196413,717.988423,6.166115e+03,...,0.165098,0.363251,0.149654,7.0,788.166667,0.0,6.0,1129.196413,717.988423,0.0
2,10385163239,Sed Et Company,"([florists supplies, nursery stock, and flower...","florists supplies, nursery stock, and flowers",a,6.61,2.519346e+04,340.452204,201.963247,2.342538e+04,...,0.622130,2.555144,0.577294,132.0,797.507171,0.0,73.0,345.115933,203.496326,3.0
3,10648956813,Proin Nisl Institute,"[(computeRs, computer peripheral equipment, an...","computers, computer peripheral equipment, and ...",a,6.66,1.419600e+06,64.583063,22.112070,1.329626e+06,...,2.702184,146.127248,2.525658,2208.0,825.551750,1574.0,14380.0,98.720466,57.083817,710.0
4,10714068705,Sollicitudin Commodo Ipsum Industries,"([furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",c,2.51,4.355100e+05,126.675378,104.490307,4.067113e+05,...,0.270890,16.845635,0.252372,1715.0,796.782831,10.0,3221.0,135.209547,113.737044,145.0


In [7]:
merchant_df.columns

Index(['merchant_abn', 'name', 'tags', 'tag', 'revenue_level', 'take_rate',
       'tot_dollar_value', 'avg_dollar_value', 'stddev_dollar_value',
       'tot_discounted_value', 'avg_discounted_value',
       'stddev_discounted_value', 'num_order_id', 'num_fraud_order',
       'rate_fraud_order', 'avg_tot_dollar_value_monthly',
       'stddev_tot_dollar_value_monthly', 'avg_tot_discounted_value_monthly',
       'stddev_tot_discounted_value_monthly', 'avg_num_order_id_monthly',
       'stddev_num_order_id_monthly', 'avg_num_fraud_order_monthly',
       'avg_tot_dollar_value_daily', 'stddev_tot_dollar_value_daily',
       'avg_tot_discounted_value_daily', 'stddev_tot_discounted_value_daily',
       'avg_num_order_id_daily', 'stddev_num_order_id_daily',
       'avg_num_fraud_order_daily', 'commission_tot_dollar_value',
       'commission_avg_dollar_value', 'commission_stddev_dollar_value',
       'commission_tot_discounted_value', 'commission_avg_discounted_value',
       'commission_stdde

In [8]:
len(merchant_df.columns)

50

In [9]:
rank_cols_desc = [
    # exposure & familiarity features
    'numd_sa2_code', 'returning_customers', 'unique_customers',
    # risk of defaulting on payments
    'median_weekly_income', 
    # financial performance
    'commission_avg_tot_dollar_value_monthly', 'avg_num_order_id_monthly',
    # fraud
    'avg_discounted_value',
]

rank_cols_asc = [
    # financial performance
    'stddev_tot_dollar_value_monthly',
    # Fraud
    'rate_fraud_order', 'stddev_tot_discounted_value_daily',
]

cols_to_keep = ['merchant_abn', 'name', 'tags', 'tag'] \
    + rank_cols_desc + rank_cols_asc

merchant_df = merchant_df[cols_to_keep]

# define the columns to add ranks for
rank_cols_dict = {}
for col in rank_cols_desc:
    rank_cols_dict[col] = False
for col in rank_cols_asc:
    rank_cols_dict[col] = True

# iterate through and rank these columns
for colname, asc in rank_cols_dict.items():
    merchant_df = RANK.add_column_rank(merchant_df, colname, ascending=asc,
        rank_type = 'minmax')

In [10]:
# calculate the uwar for the cols to rank
merchant_df = RANK.average_rank(merchant_df, rank_cols_dict.keys(),
    rank_type = 'minmax')

In [11]:
merchant_df = merchant_df.sort_values('average_rank').reset_index(drop = True)
merchant_df.index = 1 + merchant_df.index
merchant_df.index.name='rank'
merchant_df

,merchant_abn,name,tags,tag,numd_sa2_code,returning_customers,unique_customers,median_weekly_income,commission_avg_tot_dollar_value_monthly,avg_num_order_id_monthly,...,minmax_returning_customers,minmax_unique_customers,minmax_median_weekly_income,minmax_commission_avg_tot_dollar_value_monthly,minmax_avg_num_order_id_monthly,minmax_avg_discounted_value,minmax_stddev_tot_dollar_value_monthly,minmax_rate_fraud_order,minmax_stddev_tot_discounted_value_daily,average_rank
rank,,,,,,,,,,,,,,,,,,,,,
1,49891706470,Non Vestibulum Industries,"((teNt and awning shops), (a), (take rate: 5.80))",tent and awning shops,2222.0,24037.0,24078.0,815.703894,19807.260486,11786.952381,...,0.001288,0.000125,0.949333,0.333000,0.139010,0.989872,0.736454,0.012217,0.531271,0.369257
2,63290521567,Vehicula Pellentesque Corporation,"((artist supply and craft shops), (a), (take r...",artist supply and craft shops,2222.0,23346.0,24058.0,815.663697,13544.646461,8015.666667,...,0.029998,0.000955,0.949344,0.543891,0.414487,0.991384,0.454313,0.011242,0.341656,0.373727
3,45629217853,Lacus Consulting,"[[gift, Card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",2222.0,23944.0,24080.0,815.679442,27867.742923,10317.761905,...,0.005152,0.000042,0.949340,0.061567,0.246329,0.984823,0.878715,0.016873,0.655372,0.379821
4,89726005175,Est Nunc Consulting,"((tent and awning shops), (a), (take rate: 6.01))",tent and awning shops,2222.0,23930.0,24079.0,815.688162,25495.901715,10280.428571,...,0.005734,0.000083,0.949338,0.141438,0.249056,0.983478,0.917480,0.018450,0.666423,0.393148
5,64203420245,Pede Nonummy Corp.,"((tent and awning shops), (c), (take rate: 2.86))",tent and awning shops,2222.0,24055.0,24081.0,815.686432,10281.625409,12418.095238,...,0.000540,0.000000,0.949338,0.653771,0.092908,0.989884,0.776352,0.012593,0.556897,0.403228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,72064300998,Fringilla Ornare Corporation,"[(telecom), (b), (take rate: 4.85)]",telecom,NaN,NaN,NaN,NaN,0.000000,0.000000,...,NaN,NaN,NaN,1.000000,1.000000,NaN,0.000000,NaN,0.000000,NaN
2539,70814127470,Id Ante Nunc Ltd,"((telecom), (a), (take rate: 5.78))",telecom,NaN,NaN,NaN,NaN,0.000000,0.000000,...,NaN,NaN,NaN,1.000000,1.000000,NaN,0.000000,NaN,0.000000,NaN
2540,89618228232,Molestie Tortor Consulting,"([telecom], [a], [take rate: 6.24])",telecom,NaN,NaN,NaN,NaN,0.000000,0.000000,...,NaN,NaN,NaN,1.000000,1.000000,NaN,0.000000,NaN,0.000000,NaN


In [12]:
merchant_df.to_csv('../models/merchant_rankings.csv')

In [13]:
# choose the first 100 of these
final_100_df = merchant_df.head(100)
final_100_df.tail(20)

,merchant_abn,name,tags,tag,numd_sa2_code,returning_customers,unique_customers,median_weekly_income,commission_avg_tot_dollar_value_monthly,avg_num_order_id_monthly,...,minmax_returning_customers,minmax_unique_customers,minmax_median_weekly_income,minmax_commission_avg_tot_dollar_value_monthly,minmax_avg_num_order_id_monthly,minmax_avg_discounted_value,minmax_stddev_tot_dollar_value_monthly,minmax_rate_fraud_order,minmax_stddev_tot_discounted_value_daily,average_rank
rank,,,,,,,,,,,,,,,,,,,,,
81,40555823280,Nibh Donec Incorporated,"((gift, card, novelty, and souvenir shops), (a...","gift, card, novelty, and souvenir shops",2207.0,981.0,12760.0,800.177354,2888.237467,864.285714,...,0.959240,0.470141,0.953485,0.902740,0.936867,0.976180,0.108493,0.025695,0.112167,0.545176
82,51279178333,Neque LLC,"[(music shops - musical instruments, pianos, a...","music shops - musical instruments, pianos, and...",2205.0,951.0,12802.0,821.458414,618.319026,865.571429,...,0.960487,0.468397,0.947795,0.979178,0.936773,0.984648,0.074707,0.018345,0.078214,0.545620
83,76314317957,Semper Corp.,"([florists supplies, nursery stock, and flOwer...","florists supplies, nursery stock, and flowers",2210.0,3853.0,17971.0,811.431862,14073.673302,1568.761905,...,0.839912,0.253738,0.950476,0.526076,0.885408,0.926593,0.516465,0.072390,0.480984,0.545744
84,70610974780,Elit Aliquam PC,"([tent and awning shOps], [a], [take rate: 6.93])",tent and awning shops,2195.0,735.0,11669.0,809.379441,2181.507692,761.761905,...,0.969462,0.515449,0.951024,0.926539,0.944356,0.983449,0.067516,0.018986,0.068751,0.545769
85,61845622676,Turpis In Incorporated,"[[gift, card, novelty, and souvenir shops], [c...","gift, card, novelty, and souvenir shops",2214.0,1191.0,13362.0,800.932573,1257.577575,925.857143,...,0.950515,0.445141,0.953283,0.957652,0.932370,0.976468,0.110285,0.024143,0.110657,0.546412
86,36196225600,Leo Elementum Sem Associates,"[[artist supply and craft shops], [b], [take r...",artist supply and craft shops,2191.0,835.0,12282.0,800.222632,2009.372086,818.190476,...,0.965307,0.489992,0.953473,0.932335,0.940234,0.979033,0.087113,0.023317,0.085493,0.547025
87,80109801322,In Faucibus LLP,"[(cable, satellite, and other pay television a...","cable, satellite, and other pay television and...",2204.0,999.0,12839.0,823.071616,1924.892480,876.380952,...,0.958493,0.466860,0.947364,0.935180,0.935984,0.974168,0.115158,0.027826,0.113705,0.548284
88,17739089622,Auctor Quis Corp.,"((cable, satellite, and other pay television a...","cable, satellite, and other pay television and...",2196.0,983.0,12478.0,814.723029,2531.961928,847.523810,...,0.959157,0.481852,0.949596,0.914737,0.938092,0.973944,0.112100,0.027193,0.114683,0.548306
89,68004106739,Nec Ante Ltd,"((cable, satellite, and other pay television a...","cable, satellite, and other pay television and...",2189.0,1094.0,13260.0,805.685987,4049.534833,917.047619,...,0.954545,0.449377,0.952012,0.863634,0.933013,0.964037,0.158513,0.038217,0.160000,0.548821


In [14]:
# save the rank, abn, name and sales_revenue in a csv
final_100_df.to_csv('../models/top-100-merchants.csv')

In [15]:
segmented_100_df = final_100_df.join(
    segments_df.set_index('merchant_abn'),
    on = 'merchant_abn',
    how = 'left'
)

segmented_100_df.to_csv('../models/segmented_top_100.csv')

In [16]:
segmented_100_df.groupby('segment').count()

,merchant_abn,name,tags,tag,numd_sa2_code,returning_customers,unique_customers,median_weekly_income,commission_avg_tot_dollar_value_monthly,avg_num_order_id_monthly,...,minmax_returning_customers,minmax_unique_customers,minmax_median_weekly_income,minmax_commission_avg_tot_dollar_value_monthly,minmax_avg_num_order_id_monthly,minmax_avg_discounted_value,minmax_stddev_tot_dollar_value_monthly,minmax_rate_fraud_order,minmax_stddev_tot_discounted_value_daily,average_rank
segment,,,,,,,,,,,,,,,,,,,,,
gifts souvenirs,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
home furnishings,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
leisure goods and services,70,70,70,70,70,70,70,70,70,70,...,70,70,70,70,70,70,70,70,70,70
tech and telecom,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23


____

In [19]:
segmented_merchant_df = merchant_df.join(
    segments_df.set_index('merchant_abn'),
    on = 'merchant_abn',
    how = 'left'
)

In [17]:
seg_feature_names = {
    'gifts souvenirs': # revenue
        [
            'commission_avg_tot_dollar_value_monthly', 
            'avg_num_order_id_monthly',
            'stddev_tot_dollar_value_monthly',
        ],
    'home furnishings': # risk
        [
            'median_weekly_income', 
        ],
    'leisure goods and services': # exposure
        [
            'numd_sa2_code', 
            'returning_customers', 
            'unique_customers',
        ],
    'tech and telecom': # exposure
        [
            'numd_sa2_code', 
            'returning_customers', 
            'unique_customers',
        ],
}

seg_feature_weights = {}
num_features = len(rank_cols_dict.keys())

for seg, feature_list in seg_feature_names.items():
    feature_weights = []
    
    for feature in rank_cols_dict.keys():
        weight = 0.2
        if feature not in feature_list:
            weight = 1 - (0.2 * len(feature_list))
            weight /= num_features - len(feature_list)
        feature_weights.append(weight)

    print(len(rank_cols_dict.keys()))
    print(len(feature_weights))

    seg_feature_weights[seg] = feature_weights
    segmented_merchant_df = RANK.average_rank(
        segmented_merchant_df,
        rank_cols_dict.keys(),
        'minmax',
        feature_weights,
        '_'.join(seg.split(' '))
    )

10
10
10
10
10
10
10
10


In [18]:
segmented_100_df.groupby('segment').count()

,merchant_abn,name,tags,tag,numd_sa2_code,returning_customers,unique_customers,median_weekly_income,commission_avg_tot_dollar_value_monthly,avg_num_order_id_monthly,...,minmax_avg_num_order_id_monthly,minmax_avg_discounted_value,minmax_stddev_tot_dollar_value_monthly,minmax_rate_fraud_order,minmax_stddev_tot_discounted_value_daily,average_rank,average_rank_gifts_souvenirs,average_rank_home_furnishings,average_rank_leisure_goods_and_services,average_rank_tech_and_telecom
segment,,,,,,,,,,,,,,,,,,,,,
gifts souvenirs,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
home furnishings,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
leisure goods and services,70,70,70,70,70,70,70,70,70,70,...,70,70,70,70,70,70,70,70,70,70
tech and telecom,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23


In [20]:
for seg in set(segmented_merchant_df['segment']):
    print(f'\nTop 10 {seg} Merchants')
    print(
        segmented_merchant_df[
            segmented_merchant_df['segment'] == seg
        ].sort_values(
            f"average_rank_{'_'.join(seg.split(' '))}"
        ).head(10)[
            ['merchant_abn', 'name']
        ]
    )



Top 10 home furnishings Merchants


KeyError: 'average_rank_home_furnishings'

In [ ]:
for seg in set(segmented_100_df['segment']):
    print(f'\nTop 10 {seg} Merchants')
    print(
        segmented_100_df[
            segmented_100_df['segment'] == seg
        ].sort_values(
            f"average_rank_{'_'.join(seg.split(' '))}"
        ).head(10)[
            ['merchant_abn', 'name']
        ]
    )